Ce script vise à utiliser l'API Légifrance, disponible via PISTE. 

# 1/ Mise en place
Il nous faut dans un premier temps obtenir notre token : celui-ci s'obtient grâce aux identifiants générés sur le portail PISTE lorsqu'on a accepté les CGU de l'API Légifrance.

In [12]:
import requests
import pandas as pd

# Prompt the user to input client_id and client_secret
# (J'imagine que hardcoder ses identifiants est une mauvaise pratique)
client_id = input("Enter your client_id: ")  # client_id generated by PISTE
client_secret = input("Enter your client_secret: ")  # client_secret generated by PISTE

url = "https://sandbox-oauth.piste.gouv.fr/api/oauth/token"
payload = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "openid"
}
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(url, data=payload, headers=headers)

# Store the access token in a variable
access_token = None
if response.status_code == 200:
    access_token = response.json().get("access_token")
else:
    print("Error:", response.status_code, response.text)


# 2/ Accès à la documentation Swagger

In [ ]:
import json
import requests

# Charger la documentation Swagger depuis une URL
url = "https://github.com/user-attachments/files/17714249/Legifrance.json"
response = requests.get(url)
swagger_doc = response.json()

# Afficher des informations générales
print("Titre:", swagger_doc['info']['title'])
print("Version:", swagger_doc['info']['version'])
print("Description:", swagger_doc['info'].get('description', 'Pas de description disponible'))

# Parcourir les chemins (endpoints)
print("\nEndpoints disponibles :")
for path, methods in swagger_doc['paths'].items():
    print(f"\nPath : {path}")
    for method, details in methods.items():
        print(f"  Méthode : {method.upper()}")
        print("    Description :", details.get('description', 'Pas de description disponible'))

        # Afficher les paramètres de chaque méthode
        if 'parameters' in details:
            print("    Paramètres :")
            for param in details['parameters']:
                param_type = param.get('type', 'inconnu')  # Définit le type à "inconnu" si la clé "type" n'existe pas
                print(f"      - {param['name']} (type: {param_type}) - {'Obligatoire' if param.get('required') else 'Optionnel'}")

        # Afficher les réponses possibles
        print("    Réponses :")
        for status_code, response in details['responses'].items():
            print(f"      - Code {status_code}: {response.get('description', 'Pas de description')}")


# 3/ Requêtes pour les prénoms au JORF
Notre objectif, exploratoire dans un premier temps, est d'avoir pour l'année 2024 l'ensemble des nominations. Il nous faudra sans doute récupérer le NOR, qui nous renseignera le ministère et l'année. 

Pour l'instant, on voit en regardant l'output qu'il n'y a pas de tag préexistant nous facilitant beaucoup la tâche : il serait chouette que certains codes soient stables, mais j'en doute.

In [22]:
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getJoWithNor/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Afin de comprendre le format d'un output de nomination, utilisons le NOR d'une nomination donnée
payload = {
    "nor": "ECOP2427670A"
}


# Envoi de la requête
response = requests.post(url, headers=headers, json=payload)

# Vérification et affichage du résultat
if response.status_code == 200:
    data = response.json()
    print(data)  # Affiche le contenu brut de la réponse pour analyse
else:
    print(f"Erreur {response.status_code}: {response.text}")

{'executionTime': 2, 'dereferenced': False, 'id': 'JORFTEXT000050397779_01-01-2999', 'idConteneur': None, 'cid': 'JORFTEXT000050397779', 'title': "Arrêté du 18 octobre 2024 portant nominations au cabinet du ministre de l'économie, des finances et de l'industrie", 'nor': 'ECOP2427670A', 'eli': None, 'alias': None, 'jorfText': 'JORF n°0255 du 26 octobre 2024', 'jurisState': 'Sans état juridique', 'visa': "<p align='left'><br/>Le ministre de l'économie, des finances et de l'industrie,<br/>Vu le <a href='/affichTexte.do?cidTexte=JORFTEXT000034938597&categorieLien=cid' title='Décret n°2017-1098 du 14 juin 2017'>décret n° 2017-1098 du 14 juin 2017</a> relatif aux collaborateurs du Président de la République et des membres du Gouvernement ;<br/>Vu le <a href='/affichTexte.do?cidTexte=JORFTEXT000050251629&categorieLien=cid' title='Décret n°2024-892 du 23 septembre 2024'>décret n° 2024-892 du 23 septembre 2024</a> relatif à la composition des cabinets ministériels ;<br/>Vu le <a href='/affichTe

Puis, avec le module search

In [23]:
url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/search/"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}


payload = {
    "recherche": {
        "filtres": [
            {
                "valeurs": ["ARRETE"],  # Filtrer uniquement par arrêté, ce qui inclut les nominations
                "facette": "NATURE"
            },
            {
                "dates": {
                    "start": "2024-01-01",
                    "end": "2024-12-31"
                },
                "facette": "DATE_SIGNATURE"
            }
        ],
        "sort": "SIGNATURE_DATE_DESC",
        "fromAdvancedRecherche": False,
        "secondSort": "ID",
        "champs": [
            {
                "criteres": [
                    {
                        "valeur": "cabinet ministre",  # Critère pour filtrer les nominations dans les cabinets ministériels
                        "operateur": "ET",
                        "typeRecherche": "TOUS_LES_MOTS_DANS_UN_CHAMP"
                    }
                ],
                "operateur": "ET",
                "typeChamp": "TITLE"
            }
        ],
        "pageSize": 10,  # Ajustez selon vos besoins
        "operateur": "ET",
        "typePagination": "DEFAUT",
        "pageNumber": 1
    },
    "fond": "LODA_DATE"
}


# Envoi de la requête
response = requests.post(url, headers=headers, json=payload)

# Vérification et affichage du résultat
if response.status_code == 200:
    data = response.json()
    # Extraction des NOR des arrêtés
    nors = [item['nor'] for item in data.get('results', [])]
    print("NOR des nominations en 2024:", nors)
else:
    print("Erreur lors de la requête:", response.status_code, response.text)



NOR des nominations en 2024: ['MOMC2426137A', 'MOMC2429204A', 'EAEC2428687A', 'MOMC2429076A', 'PRMX2429339A', 'PRMX2429154A', 'PRMX2428951A', 'PRMX2428841A', 'MOMC2427547A', 'EAEC2427273A']
